# Introduction

What we did in this notebook is selecting a small part of the amazon dataset and use it to test the methods we implemented.

We tried to write code which makes use of Spark SQL, however due to the limitated competences of certain members of the group we weren't always able to provide efficient code. This issue will be resolved in Milestone 3.

Please also note that due to time constraints and cluster overload we weren't able to properly test our methods on the cluster.

# Import/Helpers

In [1]:
import os
import findspark
os.environ['SPARK_HOME'] = '/opt/apache-spark'
# os.environ['SPARK_HOME'] = '/usr/local/Cellar/apache-spark/2.2.0/libexec/'
findspark.init()
from pyspark import SQLContext, SparkContext, SparkConf
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.mllib.stat import Statistics as S
import json
import numpy as np
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark_dist_explore import hist
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def correlation(df, col1, col2, method='pearson'):
    rdd1 = df.select(col1).rdd.map(lambda x: x[0])
    rdd2 = df.select(col2).rdd.map(lambda x: x[0])
    return S.corr(rdd1, rdd2, method)

Creation of a Spark contest.

In [3]:
memory = '6g'
pyspark_submit_args = ' --driver-memory ' + memory + ' pyspark-shell'
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

sc = SparkContext()
sqlContext = SQLContext(sc)

# Questions

We load the data from the json file. The data is a small subset containing only items from the following categories:
"Baby", "Automotive" and "Grocery and Gourmet Food". We will use this dataset on our local machine to qualitatively manipulate the data. 

In [45]:
df_review = sqlContext.read.json("reviews_auto_baby_grocery.json")
# df_review = sqlContext.read.json("datasets/data.json")
df_review.write.parquet("data.parquet")

KeyboardInterrupt: 

In [4]:
# df_review = sqlContext.read.json("reviews_auto_baby_grocery.json")
df_review = sqlContext.read.parquet("data.parquet")

num_reviews = df_review.count()

Here we load the product's metadata.

In [11]:
# meta_df = sqlContext.read.json("meta_auto_baby_grocery.json")
meta_df = sqlContext.read.parquet("metadata.parquet")
#meta_df = sqlContext.read.json("datasets/metadata.json")

Setup for parquet file:

In [7]:
# We flatten the meta dataframe
def flatten(salesRank):
    if salesRank == None:
        return None, None
    else:
        for category, rank in salesRank.asDict().items():
            if rank != None:
                return (category, rank)
    return None, None
flatten = udf(flatten, StructType([StructField("category", StringType()), StructField("rank", IntegerType())]))
df_flattened = meta_df.withColumn("salesRank", flatten("salesRank"))
df_flattened = df_flattened.drop("_corrupt_record")
df_flattened = df_flattened.select(list(set(df_flattened.schema.names) - set(["salesRank", "related"]))+ ["salesRank.*", "related.*"])
df_flattened.write.parquet("metadata.parquet")

In [12]:
meta_df

DataFrame[description: string, categories: array<array<string>>, title: string, imUrl: string, asin: string, price: double, brand: string, category: string, rank: int, also_bought: array<string>, also_viewed: array<string>, bought_together: array<string>, buy_after_viewing: array<string>]

In [6]:
# No need to run it, already checked...

# We check that there is no metadata with two ranks
def not_one_or_none(salesRank):
    count = 0
    # If no rank, then false
    if salesRank == None:
        return 0
    for category, rank in salesRank.asDict().items():
        if rank != None:
            count += 1
    return int(count != 1 and count != 0)
not_one_or_none = udf(not_one_or_none, IntegerType())
not_one_or_none = meta_df.withColumn("exactly_one", not_one_or_none("salesRank")).agg(F.sum("exactly_one").alias("sum")).head()[0]
print("There is %.d article with not exactly one category for a rank"%not_one_or_none)

There is 0 article with not exactly one category for a rank


We filter the salesRank to have only ranked products. There is two possiblity, either the salesRank is completely none, either the categories are all None. we take care of both cases.

In [8]:
filtered_meta_df = df_flattened.dropna(subset=['rank', "category"])
filtered_meta_df.count()

6151598

In [10]:
df_flattened.count()

9430088

## Question 1: review quantity vs quality 

Here we explore some metrics related to reviews: a quantitative one - the number of words per review - and a qualitative one - the helpfulness of a review. 

### Word count
Here we count the number of word in a certain review.

In [ ]:
wordCount = F.udf(lambda text: len(text.split(" ")), T.IntegerType())

df = df_review.withColumn('wordcount', wordCount(df_review["reviewText"]))
df.select("wordcount").describe().show()
overall = df.select("overall").rdd.map(lambda x: x[0])
wordcount = df.select("wordcount").rdd.map(lambda x: x[0])
pearson_corr = correlation(df, "overall", "wordcount", method='pearson')
spearman_corr = correlation(df, "overall", "wordcount", method='spearman')
print("The pearson correlation is %.3f and the spearman is %.3f"%(pearson_corr, spearman_corr))

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(8, 5)
hist(ax, df.select("wordcount"), log=True);
pearson_corr = correlation(df.filter(df.wordcount > 1000), "overall", "wordcount", method='pearson')
spearman_corr = correlation(df.filter(df.wordcount > 1000), "overall", "wordcount", method='spearman')
print("The pearson correlation is %.3f and the spearman is %.3f"%(pearson_corr, spearman_corr))

### Helpfulness
We get the "helpfulness" of a review by computing the ratio of people that found a review helpful and those that voted. If we have no data, we simply put -1 as a ratio.

In [ ]:
df_review = df_review.withColumn("helpfulness", F.when(df_review.helpful.getField(1) > 0, df_review.helpful.getField(0)/df_review.helpful.getField(1)).otherwise(-1))

In [ ]:
df_review.filter(df_review["helpfulness"] >= 0.0).select("helpfulness").describe().show()

There seems to have some discrepancies amongst the ratio, since the max is at 2.0. We will check that it does not occur too often and may drop the data in the future.

In [ ]:
df_review.filter(df_review["helpfulness"] > 1.0).select("helpful").show()

Since only two reviews have problems, we will simply ignore them.

We then look at some distribution of helpfulness and rating.

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(8, 5)
hist(ax, df_review.filter((df_review["helpfulness"] >= 0.0) & (df_review["helpfulness"] <= 1.0)).select("helpfulness"));

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(8, 5)
hist(ax, df_review.filter((df_review["helpfulness"] >= 0.0) & (df_review["helpfulness"] <= 1.0)).select("overall"));

In [ ]:
correlation(df_review.filter((df_review["helpfulness"] >= 0.0) & (df_review["helpfulness"] <= 1.0)), "overall", "helpfulness")

With only these three metrics we aren't able to draw any meaningful conclusion.

Here we compute the correlation between a product ranking and -respectively - the number of reviews it has, the average of the reviews helpfulness, the average of the reviews ranking.

In [ ]:
output_df = filtered_meta_df[['asin', 'unique_rank']]

joined_df = filtered_meta_df.join(df_review, 'asin')
grouped_df = joined_df.groupBy('asin')

review_count_df = grouped_df.count()
mean_helpfulness_df = grouped_df.mean('helpfulness', 'overall')
output_df = output_df.join(review_count_df, 'asin').join(mean_helpfulness_df, 'asin')
rank_count_correlation = output_df.corr('unique_rank', 'count', method='pearson')
rank_helpfulness_correlation = output_df.corr('unique_rank', 'avg(helpfulness)', method='pearson')
rank_overall_correlation = output_df.corr('unique_rank', 'avg(overall)', method='pearson')
print(rank_count_correlation, rank_helpfulness_correlation, rank_overall_correlation)

## Question 2: influence of reviewer personality

We get the count of review of distinct article per reviewer and filter those that have less than 5 review.

In [ ]:
df_count = df_review.groupBy("reviewerID").agg(F.countDistinct("asin").alias('count')).filter("`count` >= 5")

We get all reviews of reviewers with more than 5 articles.

In [ ]:
df_filtered = df_review.join(df_count, on='reviewerID').drop('count')
num_five_reviews = df_filtered.count()
print("We have %.d, i.e. %.3f%% of review who belongs to reviewer with 5 or more reviews"%(num_five_reviews, num_five_reviews/num_reviews))

We compute the average grade for each reviewer.

In [ ]:
average_std_score = df_filtered.groupBy("reviewerID").agg(F.avg("overall").alias("mean"), F.stddev("overall").alias("std"))
average_std_score.cache().count()

Here are some statistic concerning the average grade for each reviewer.

The first row contains the total number of reviewer considered.

The second row contains the mean of the reviewers' averages grades (left) and the standard deviation of said mean (right).

The third row contains the mean of the reviewers' standard deviations (left) and the standard deviation of said mean (right).

The fourth row contains the mean of the reviewer's lower grades (left) and the standard deviation of said mean (right).

The fourth row contains the mean of the reviewer's higher grades (left) and the standard deviation of said mean (right).

In [ ]:
average_std_score.select("mean", "std").describe().show()

We then plot the average grade for each reviewer through histograms, with the grade on the x-axys and the number of reviewers who have such mean ranking on the y-axis.

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(8, 5)
hist(ax, average_std_score.select("mean"));

It can be noted that reviewers tend to give high ratings.

We then plot the average standard deviation for each reviewer through histograms, with the grade on the x-axys and the number of reviewers who have such average standard deviation on the y-axis.

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(8, 5)
hist(ax, average_std_score.select("std"));

It can be noted that the majority of reviewers tend to have either a very small standard deviation (0-0.25) or a standard deviation between 0.5 and 1.5. The number of people with an higher standard deviation are sensibily less.

Here, we look at the mean of the ratings given by reviewer with small (<0.25) standard deviation.
We also look at what percentage of reviewer they correspond.

In [ ]:
print("There is %.2f%% of reviewer with std < 0.25"%(average_std_score.filter(average_std_score.std < 0.25).count()/average_std_score.count()*100))
fig, ax = plt.subplots()
fig.set_size_inches(8, 5)
hist(ax, average_std_score.filter(average_std_score.std < 0.25).select("mean"));

It can be noted that people with a small standard deviation, i.e. people who tend to always give the same rating, tend to give only very high rating (>4.5).

In conclusion, we have hints that there's a certain bias related to reviewers' personality and habits. 
The fact that the mean of the average reviewer rating is so high for example could make us think that people tend to rate the product they're satisfied with and not those they're unsastisfied with.
The fact that the reviewers with little standard deviation tend to give high ratings also speaks in that direction.

## Question 3: Brand fidelity/hate

We load the metadata datas and only keep the article number and the brand that are not null, since we are interested in brands.

We then join the dataframe of reviews with the metadata and display some stats.

In [ ]:
df_meta = sqlContext.read.json("meta_auto_baby_grocery.json")
df_meta_brand = df_meta.select("asin", "brand").dropna()

df_with_brand = df_filtered.join(df_meta_brand, on='asin')
num_reviews_with_brand = df_with_brand.count()

print("We have %.d, which represents %.3f%% of review with a brand and who belongs to reviewer with 5 or more reviews"%(num_reviews_with_brand, num_reviews_with_brand/num_reviews))
print("We have %.3f%% of review amongst those who belongs to reviewer with 5 or more reviews that have a brand "%(num_reviews_with_brand/num_five_reviews))

We can see that once again we loose a lot of reviews with this approach, but it is still an interesting number to work with.

Then we group by brand and reviewer, compute some stats and filter out all reviewers that have less than 5 reviews for a specific brand.

In [ ]:
df_brand_reviewer = df_with_brand.groupBy(["brand", "reviewerID"]).agg(F.count("overall").alias("count"), F.avg("overall").alias("mean"), F.stddev("overall").alias("std")).filter("`count` >= 5")
brand_reviewers_count = df_brand_reviewer.cache().count()
print("We have %.d brand/reviewers tuples"%(brand_reviewers_count))

We then show some stats about the mean and std.

In [ ]:
df_brand_reviewer.select("count", "mean", "std").describe().show()
fig, ax = plt.subplots()
fig.set_size_inches(8, 5)
hist(ax, df_brand_reviewer.select("mean"));
fig, ax = plt.subplots()
fig.set_size_inches(8, 5)
hist(ax, df_brand_reviewer.select("std"));

By comparing qualitatively (we will do a more precise analysis in milestone 3) those graphs with the one from Question 2, we can see that the ratings seems to be having less deviation, showing a tendency to love/hate toward a brand.

We try to see if there is an obvious correlation between number of reviews for a brand and the mean rate:

In [ ]:
pearson_corr = correlation(df_brand_reviewer, "count", "mean", method='pearson')
spearman_corr = correlation(df_brand_reviewer, "count", "mean", method='spearman')
print("The pearson correlation is %.3f and the spearman is %.3f"%(pearson_corr, spearman_corr))

At first sight, we cannot see an obvious relationship, however, we will do a more refined analysis in the next milestone.

Compute the total number of products.

In [ ]:
total_products = meta_df.count()
total_products

Compute the ratio between ranked and total products.

In [ ]:
ranked_products = filtered_meta_df.count()
ranked_products
ratio = ranked_products/total_products
print("ranked_products: " + str(ranked_products) + " total products: " + str(total_products) + " ratio: " + str(ratio))

We can notice that in this restricted dataset less than one third of the products is ranked, a fact that makes our rank-based analysis less representative. 

We now check that there is exactly one category that has a sales rank.

Here we obtain the names of the ranking categories.

In [ ]:
rank_categories = filtered_meta_df.head().salesRank.asDict().keys()

## Question 4: influence of 'also bought' feature

I'll begin by focus on the 'bought together' feature and not on the 'also bought' feature right away.
This both because the former is more relevant to the question and because it scales better: 'also bought' tends to be a very long list of products while 'bought together' is usually much shorter and easier to analyze.

In order to answer question 4, it is possible to use the following approach:
1. Find the best ranked products for each category.
2. look at the rank of the products that were bought together with them.
3. draw  qualitative conclusions.

First we select the interesting columns of the metadata dataframe, flattening the hierarchical datastructure in the process.

Then we define a function that allows to find in which category a product has been ranked.

In [43]:
#same analysis for bought_together!
q4_meta_df = filtered_meta_df.select("asin", "related.*", "unique_rank")
q4_meta_df = q4_meta_df.select("asin", "unique_rank", F.explode("bought_together").alias("paired_product")).dropna(how='any')
q4_joined_meta_df = q4_meta_df.alias("df1").join(q4_meta_df.alias("df2"), F.col("df1.paired_product") == F.col("df2.asin"), "inner")
q4_joined_meta_df.first()

Row(asin='B0000D94P1', unique_rank=118616, paired_product='B0000D94PL', asin='B0000D94PL', unique_rank=129998, paired_product='B0000D94P1')

It can be noted that products with very different rankings were bought together.
This means that question 4 might indeed make sense, there might be "good" product that improve the sales of "bad" products. As of now "good" means "popular" and bad means "unpopolar", but the definition might change when we delve even further into the data.

## Question 5: rank/review ratio

The approach for question 5 is the following:
1. For each rank category, take the 5 best ranked products
2. look in the review table and see how many review were made for each product
3. Print the values and try to qualitatevely interpet. 

Here we apply the described pipeline.

In [ ]:
for category in rank_categories:
    category_df = filtered_meta_df.select(category, "asin").dropna().sort(category, ascending = False)
    best_products = category_df.take(5)
    for index, best_product in enumerate(best_products):
        productid = best_product.asin
        rank = best_product[category]
        number_reviews = df_review.filter(review_df.asin == productid).count()
        print("There are " + str(number_reviews) + " reviews for the product ranked " + str(rank) + " in the category " + category)

There's doesn't seem to be any correlation between the number of reviews associated to a product and its sales ranking according to these results, which don't hint at a linear relationship between number of reviews and sales.

## Question 6: product categorization

I start by filtering the metadata to drop the uncategorized products.

In [ ]:
filtered_meta_df = meta_df.dropna(subset=['categories'])
filtered_meta_df.count()

Then I proceed to analyze the products.
The approach is the following:
1. I create a dictionary to map each category to the number of reviews written for that category.
2. for each product, I extract the categories it is part of.
3. I compute the number of reviews for that product and I update the values associated to the product categories in the dictionary.

I limit the analysis to 10 products on my local machine due to the heavy computational cost, however on the cluster we'll analyze all of them.

In [ ]:
category_dict = {}
#NOTE: the commented line is the one to be used in the cluster.
#for product in filtered_meta_df.collect():
for product in filtered_meta_df.take(10):
    productid = product.asin
    categories = product.categories[0]
    for category in categories:
        number_reviews = review_df.filter(review_df.asin == productid).count()
        old_value = category_dict.get(category, 0)
        category_dict[category] = old_value + number_reviews

print(str(category_dict))

This small and potentially unrepresentative dataset shows there's a wide difference in the number of reviews between categories. To say more we need the results from the cluster, which we couldn't obtain due to time constraint and cluster overload.

# TO DO FOR MILESTONE 3 

## Question 1 

Find metrics allowing to draw meaningful conclusions.

## Question 2

Better quantify the bias of the reviewer by analysing other aspects 

## Question 3

Further delve into the data to produce a coherent conclusion.

## Question 4

Further delve into the data and formally describe the relation between the 'also bought' and 'bought together' features as well as the meaning of a "good" and "bad" product, which as of now is associated to popularity. 
Also substitute the print based approach with a more complex one.

Draw formally correct conclusion with regard to the influence of the 'also_bought' feature on sales.

## Question 5

Confirm the conclusion by working on the whole dataset through the cluster and substitute the print based approach with a more complex one. 

## Question 6 

Confirm the conclusion by working on the whole dataset through the cluster and substitute the dictionary based approach with a more complex one. 

## Refine analysis

Eventually find new question related to reviews and answer them.

## Final Conclusion

produce a clear description of the characteristic of an amazon review: how can review be in interpreted in the complex system that is Amazon?

## Report

sum up the results of the research in a 4-page report.

## Poster

design a poster to present the results of our research.